In [ ]:
import pandas as pd

In [ ]:
diet_csv = pd.read_csv("diet.csv").copy()
recipes_csv = pd.read_csv("recipes.csv").copy()
requests_csv = pd.read_csv("requests.csv").copy()
reviews_csv = pd.read_csv("reviews.csv").copy()
not_vegetarian_input = pd.read_excel("not_vegetarian.xlsx", header=None, names=["Sheet1"])
not_vegan_input = pd.read_excel("not_vegan.xlsx", header=None, names=["Sheet1"])

In [ ]:
# merge diet + request
request_with_diet = pd.merge(diet_csv, requests_csv, how="inner", on="AuthorId")
# merge diet + request + recipe
request_with_diet_and_recipe = pd.merge(recipes_csv, request_with_diet, how="inner", on="RecipeId")
# merge diet + request + recipe + review
df = pd.merge(reviews_csv, request_with_diet_and_recipe, how="inner", on=["AuthorId", "RecipeId"])

In [ ]:
import openpyxl
df["RecipeIngredientParts"] = df["RecipeIngredientParts"].str.replace(")", '')
df["RecipeIngredientParts"] = df["RecipeIngredientParts"].str.replace("(", '')
df["RecipeIngredientParts"] = df["RecipeIngredientParts"].str.replace("\"", '')
df["RecipeIngredientParts"] = df["RecipeIngredientParts"].str.replace("\\", '')
df['RecipeIngredientParts'] = df['RecipeIngredientParts'].str.replace('^c', '', regex=True)

# Extract ingredients from the "RecipeIngredientParts" column and concatenate them
all_ingredients = ','.join(df['RecipeIngredientParts'])

# Split the concatenated string into a list of individual ingredients
individual_ingredients = all_ingredients.split(',')

# Convert the list to a set to get distinct ingredients
distinct_ingredients = set(individual_ingredients)

# Convert the set to a list if needed
distinct_ingredients_list = list(distinct_ingredients)

omnivore_list = []
vegan_list = []
vegetarian_list = []
for items in distinct_ingredients_list:
    if str(items).__contains__("meat") | str(items).__contains__("chicken") | str(items).__contains__("lamb") | str(items).__contains__("beef") | str(items).__contains__("pork") | str(items).__contains__("bacon") | str(items).__contains__("fish") | str(items).__contains__("sausage") | str(items).__contains__("turkey") | str(items).__contains__("tuna") \
            | str(items).__contains__("salmon") | str(items).__contains__("trout") :
        omnivore_list.append(items)
        distinct_ingredients_list.remove(items)
    elif str(items).__contains__("cheese") | str(items).__contains__("honey") | str(items).__contains__("mozzarella") |str(items).__contains__("parmesan"):
        omnivore_list.append(items)
        vegetarian_list.append(items)
        distinct_ingredients_list.remove(items)
    
# read in lists from outside:
omnivore_df = pd.read_excel("Machine_Learning_Vegetarisch.xlsx", header=None, names=["1373"])
vegetarian_df = pd.read_excel("Machine_Learning_Vegan_1_1.xlsx", header=None, names=["1373"])

omnivore_outside = []
omnivore_outside.extend(omnivore_df["1373"].tolist())
omnivore_list.extend(omnivore_outside)

vegetarian_outside = []
vegetarian_outside.extend(vegetarian_df["1373"].tolist())
vegetarian_list.extend(vegetarian_outside)

df_not_vegetarian = pd.DataFrame(omnivore_list)
df_not_vegan = pd.DataFrame(vegetarian_list)

df_not_vegetarian.to_excel("not_vegetarian.xlsx")
df_not_vegan.to_excel("not_vegan.xlsx")









not_vegetarian_list = []
not_vegan_list = []

not_vegetarian_list.extend(not_vegetarian_input["Sheet1"].tolist())
not_vegan_list.extend(not_vegan_input["Sheet1"].tolist())

print(not_vegetarian_list)
df["DietMatchesRecipe"] = True

for index,rows in df.iterrows():
    if rows["Diet"] == "Omnivore":
        continue
    elif rows["Diet"] == "Vegetarian":
        for ingredients in not_vegetarian_list:
            if ingredients in df.at[index, "RecipeIngredientParts"]:
                df.at[index, "DietMatchesRecipe"] = False
                break
    elif rows["Diet"] == "Vegan":
        for ingredients in not_vegan_list:
            if ingredients in df.at[index, "RecipeIngredientParts"]:
                df.at[index, "DietMatchesRecipe"] = False
                break


selected_features =["DietMatchesRecipe", "AuthorId", "RecipeId"]
diet_fit_output = df[selected_features]
diet_fit_output.to_csv("DietMatchesRecipe.csv", index=False)